# Tutorial on How to Deploy Your KG UI

### #1. Get Access to Dokku

Dokku is an ssh server, and in order to deploy your app you will need to run commands on the server (ssh and git push). 

Follow these steps to get set up with Dokku: https://dokku.dev.maayanlab.cloud/user/10-access.html. Note that you will need to request access from Daniel Clarke and provide him with your GitHub handle to run dokku commands.

### #2. Create a Dump of your Neo4j Database


First make sure your container is running on Neo4j. We will need to stop the container in order to create the dump of your Neo4j db.

In [ ]:
docker-compose stop neo4j
docker-compose run neo4j neo4j-admin database dump neo4j
docker-compose start neo4j

This creates the dump in binary located at "KnowledgeGraphDemo/data/neo4j/dumps/neo4j.dump"

### #3. Create Neo4j on Dokku

In [ ]:
dokku neo4j:create [db-name]

### #4. Ingest your Dump

In [ ]:
cd data/neo4j/dumps
dokku neo4j:import [db-name] < neo4j.dump

Now your Neo4j dump has been sent to the dokku database.

### #5. Create your App

In [ ]:
dokku apps:create [app-name]

### #6. Get the Credentials for your App and Update your .env File in Gene-Knowledge-Graph

In [ ]:
dokku neo4j:info [app-name] --dsn

This will return an output in this format: bolt://<span style="background-color: #FFFF00">[new_username]</span>:<span style="background-color: #FFFF00">[new_password]</span>@<span style="background-color: #FFFF00">[new_url]</span>

Update your .env file in Gene-Knowledge-Graph like so

NEXT_PUBLIC_PREFIX=\
NEXT_PUBLIC_NEO4J_DEV_URL=bolt://localhost:7687\
NEXT_PUBLIC_NEO4J_URL=bolt://<span style="background-color: #FFFF00">[new_url]</span>\
NEXT_PUBLIC_NEO4J_V5_URL=bolt://<span style="background-color: #FFFF00">[new_url]</span>\
NEO4J_DEV_URL=bolt://localhost:7687

NEXT_PUBLIC_NEO4J_USER=<span style="background-color: #FFFF00">[new_username]</span>\
NEXT_PUBLIC_NEO4J_NAME=kg


NEXT_PUBLIC_NEO4J_PASSWORD=<span style="background-color: #FFFF00">[new_password]</span>



NEXT_PUBLIC_DOMAINS=minio.dokku.maayanlab.cloud,github.com,minio.dev.maayanlab.cloud,s3.amazonaws.com,raw.githubusercontent.com
NEXT_PUBLIC_ENRICHR_URL=https://maayanlab.cloud/Enrichr \
NEXT_PUBLIC_GENESHOT_URL=https://maayanlab.cloud/geneshot \
NEXT_PUBLIC_HOST=https://<span style="background-color: #FFFF00">[app_name]</span>.dev.maayanlab.cloud/ \
NEXT_PUBLIC_SCHEMA=\
NEXT_PUBLIC_TURL=turlkey\
NEXT_PUBLIC_TURL_URL=https://maayanlab.cloud/turl/api/register \
APP_NAME=<span style="background-color: #FFFF00">[app_name]</span>\
DOCKERHUB_NAME=<span style="background-color: #FFFF00">[dockerhub_name]</span>\
NEO4J_VERSION=5
NEO4J_USER=

### #6. Update Dokku Config

You will need to run this command: dokku config:set [app_name] "SOME_KEY=SOME_VALUE", but will need to add EVERY key-value pair in your .env file in quotes.

For example if we wanted to update the config with the first three key-value pairs, the command would look like this:

In [ ]:
dokku config:set [app_name] "NEXT_PUBLIC_NEO4J_DEV_URL=bolt://localhost:7687" 
"NEXT_PUBLIC_NEO4J_URL=bolt://[new_url]" "NEXT_PUBLIC_NEO4J_V5_URL=bolt://[new_url]"

If you want to review the config, run:

In [ ]:
dokku config:show [app_name]

And if you accidentally set a wrong key/value pair, run:

In [ ]:
dokku config:unset my-app SOME_KEY // *Note that there's no quotes around SOME_KEY

### #7. Build the Site

In [ ]:
cd Gene-Knowledge-Graph
npm i
npm version patch

### #8. Deploy Directly from DockerHub

In [ ]:
dokku git:from-image [app_name] [image_name]

Check docker-compose.yml in Gene-Knowledge-Graph and look for "services>kg>image" for the image name. Make sure to include the version at the end of the image. This version will update each time you run "npm version patch" and rebuild, so make sure to update the git command as well after each time you rebuild.

### #9. Finalizing Deployment

In [ ]:
dokku proxy:clear-config [app_name]
dokku proxy:ports-set [app_name] 'http:80:<port_number>' // Ex. 3000 if your local site was at localhost:3000
dokku traefik:enable [app_name]
dokku neo4j:link [db_name] [app_name]

And that's all! Your site is deployed at https://<span style="background-color: #FFFF00">[app_name]</span>.dev.maayanlab.cloud/!!

### #10. What to do if you Make Changes and Want to Rebuild the App

In [ ]:
cd Gene-Knowledge-Graph
npm version patch
dokku git:from-image [app_name] [image_name] // remember to get the new version from docker-compose.yml

### #11. Maintenance

Reference the User Guide for some commands that will help you manage your app: https://dokku.dev.maayanlab.cloud/user/40-manage.html.